# Rep 2. Checkpoint 4. Value at Risk con EWMA

# Eduardo Veytia Hernández

### Actividad — Simulación Monte Carlo con portafolio real
### Esta actividad es necesaria para la entrega del Reporte Integrador 2.

 

### Usando el portafolio elegido por ti en la actividad Rep 2. Checkpoint 3. Calcular VaR (%) para tu portafolio a 15 días al 95% de confianza con los siguientes procesos.

- Con MGB con volatilidad constante (como en el checkpoint 3.)
- Usando 1 año de data histórica
- Usando 3 años de data histórica
- Usando 5 años de data histórica
- Usando 10 años de data histórica
- Con EWMA (como visto en clase)
- Usando 1 año de data histórica. Lambda =.94
- Usando 3 años de data histórica. Lambda =.94
- Usando 5 años de data histórica. Lambda =.94
- Usando 10 años de data histórica. Lambda =.94
- Con EWMA (como visto en clase) 
- Usando 1 año de data histórica. Lambda =.8
- Usando 3 años de data histórica. Lambda =.8
- Usando 5 años de data histórica. Lambda =.8
- Usando 10 años de data histórica. Lambda =.8
Construye una tabla comparativa, dentro de tu codigo, con los resultados observados.


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt

In [3]:
# Descarga de datos
stocks = ["AAPL","MSFT","GOOGL","AMZN"]
weights = np.array([0.25,0.25,0.25,0.25])

end = dt.datetime.today()
start = end - dt.timedelta(days=365*3)

data = yf.download(stocks,start=start,end=end)["Close"]
returns = data.pct_change().dropna()

n_sim=10000
n_assets=len(stocks)
n_days=252

[*********************100%***********************]  4 of 4 completed

1 Failed download:
['MSFT']: TypeError("'NoneType' object is not subscriptable")
C:\Users\eduve\AppData\Local\Temp\ipykernel_19904\1776363931.py:9: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change().dropna()


In [4]:
# Parámetros VaR
num_sim = 10000
horizon = 15
alpha = 0.05

years_list = [1, 3, 5, 10]  # ventanas históricas

end = dt.datetime.now()

In [5]:

def ewma_cov_matrix(returns_df, lam=0.94):
    """
    returns_df: DataFrame (T x k) de retornos diarios
    EWMA: S_t = lam*S_{t-1} + (1-lam)*r_{t-1}r_{t-1}'
    """
    R = returns_df.values
    T, k = R.shape

    S = np.cov(R, rowvar=False)  # inicial: cov muestral

    for t in range(1, T):
        r = R[t-1].reshape(k,1)
        S = lam*S + (1-lam)*(r @ r.T)

    return S

def simulate_var_mgb(mean_returns, cov_matrix, weights, horizon=15, num_sim=5000, alpha=0.05):
    """
    Simula como tu notebook:
    Z ~ N(0,1), correlaciona con Cholesky, suma media, port_daily, cumprod.
    VaR 95% = 1 - q_5% del valor final (pérdida positiva)
    """
    k = len(weights)
    L = np.linalg.cholesky(cov_matrix)

    final_values = np.zeros(num_sim)

    for m in range(num_sim):
        Z = np.random.normal(size=(horizon, k))
        correlated = Z @ L.T
        daily_returns = correlated + mean_returns.values
        port_daily = daily_returns @ weights
        path = np.cumprod(1 + port_daily)
        final_values[m] = path[-1]

    q05 = np.quantile(final_values, alpha)
    var_95 = 1 - q05  # pérdida como proporción

    return var_95, final_values

In [6]:
results_mgb = []

for years in years_list:
    start = end - dt.timedelta(days=365*years)

    prices = yf.download(stocks, start=start, end=end, progress=False)["Close"]
    returns = prices.pct_change().dropna()

    mean_returns = returns.mean()
    cov_matrix = returns.cov()

    # Simulación + VaR
    var_95, final_values = simulate_var_mgb(mean_returns, cov_matrix, weights,
                                            horizon=horizon, num_sim=num_sim, alpha=alpha)

    results_mgb.append([years, var_95])

df_mgb = pd.DataFrame(results_mgb, columns=["Años Data", "VaR 15d 95% (MGB const)"])
df_mgb

,Años Data,VaR 15d 95% (MGB const)
0,1,0.086542
1,3,0.068541
2,5,0.088904
3,10,0.080898


In [7]:
results_ewma94 = []

for years in years_list:
    start = end - dt.timedelta(days=365*years)

    prices = yf.download(stocks, start=start, end=end, progress=False)["Close"]
    returns = prices.pct_change().dropna()

    mean_returns = returns.mean()
    cov_ewma = ewma_cov_matrix(returns, lam=0.94)

    var_95, _ = simulate_var_mgb(mean_returns, cov_ewma, weights,
                                 horizon=horizon, num_sim=num_sim, alpha=alpha)

    results_ewma94.append([years, var_95])

df_ewma94 = pd.DataFrame(results_ewma94, columns=["Años Data", "VaR 15d 95% (EWMA λ=0.94)"])
df_ewma94

,Años Data,VaR 15d 95% (EWMA λ=0.94)
0,1,0.066675
1,3,0.058247
2,5,0.065053
3,10,0.061446


In [8]:
results_ewma8 = []

for years in years_list:
    start = end - dt.timedelta(days=365*years)

    prices = yf.download(stocks, start=start, end=end, progress=False)["Close"]
    returns = prices.pct_change().dropna()

    mean_returns = returns.mean()
    cov_ewma = ewma_cov_matrix(returns, lam=0.8)

    var_95, _ = simulate_var_mgb(mean_returns, cov_ewma, weights,
                                 horizon=horizon, num_sim=num_sim, alpha=alpha)

    results_ewma8.append([years, var_95])

df_ewma8 = pd.DataFrame(results_ewma8, columns=["Años Data", "VaR 15d 95% (EWMA λ=0.8)"])
df_ewma8

,Años Data,VaR 15d 95% (EWMA λ=0.8)
0,1,0.064892
1,3,0.056973
2,5,0.064275
3,10,0.060167


In [9]:
df_comp = df_mgb.merge(df_ewma94, on="Años Data").merge(df_ewma8, on="Años Data")
df_comp

,Años Data,VaR 15d 95% (MGB const),VaR 15d 95% (EWMA λ=0.94),VaR 15d 95% (EWMA λ=0.8)
0,1,0.086542,0.066675,0.064892
1,3,0.068541,0.058247,0.056973
2,5,0.088904,0.065053,0.064275
3,10,0.080898,0.061446,0.060167
